In [2]:
!pip install -q unsloth accelerate bitsandbytes xformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    use_gradient_checkpointing = True,
)

# ✅ Attach LoRA adapters WITHOUT task_type
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
import json
from datasets import Dataset

# Load dataset
with open("creativity_dataset_200.json", "r") as f:
    data = json.load(f)

# Convert to HuggingFace Dataset format
hf_data = Dataset.from_list([
    {
        "prompt": f"{sample['instruction']}\n\n{sample['input']}",
        "completion": sample["output"]
    } for sample in data
])


In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Create training args
training_args = TrainingArguments(
    output_dir = "./mistral-creativity-model",
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    num_train_epochs = 50,
    learning_rate = 2e-5,
    bf16 = torch.cuda.is_bf16_supported(),
    fp16 = torch.cuda.is_available(),
    logging_steps = 10,
    save_steps = 100,
    save_total_limit = 2,
    report_to = "none"
)

# Prepare model and trainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = hf_data,
    dataset_text_field = "prompt",
    max_seq_length = 2048,
    args = training_args
)

# Start fine-tuning
trainer.train()


Unsloth: Tokenizing ["prompt"] (num_proc=2):   0%|          | 0/200 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 200 | Num Epochs = 50 | Total steps = 1,250
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/7,000,000,000 (0.60% trained)


Step,Training Loss
10,0.379100
20,0.346200
30,0.323900
40,0.305200
50,0.305800
60,0.278900
70,0.263900
80,0.238700
90,0.223400
100,0.189900


TrainOutput(global_step=1250, training_loss=0.10257671856880188, metrics={'train_runtime': 4043.2029, 'train_samples_per_second': 2.473, 'train_steps_per_second': 0.309, 'total_flos': 2.261522907075379e+16, 'train_loss': 0.10257671856880188})

In [6]:
trainer.model.save_pretrained("./mistral-creativity-finetuned")
tokenizer.save_pretrained("./mistral-creativity-finetuned")


('./mistral-creativity-finetuned/tokenizer_config.json',
 './mistral-creativity-finetuned/special_tokens_map.json',
 './mistral-creativity-finetuned/tokenizer.model',
 './mistral-creativity-finetuned/added_tokens.json',
 './mistral-creativity-finetuned/tokenizer.json')

In [7]:
def generate_response(prompt, max_tokens=200):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Post-process to match desired format
    # Example format:
    # "Correctness: 1\nCreativity: 7\nReasoning: The child creatively used a drawing to solve the problem."
    correctness = 0  # Default to incorrect answer
    if "correct" in response.lower():
        correctness = 1  # If the model considers it correct, update to 1

    # Set creativity based on analysis or just hardcode for now (7 in your case)
    creativity = 7  # This can be adjusted dynamically based on content analysis

    # Reasoning about the problem
    reasoning = response.strip()

    # Adjust reasoning for more detailed output
    if "error" in reasoning.lower():
        correctness = 0  # Reassure correctness is marked as 0 if error is detected

    processed_response = f"Correctness: {correctness}\n"
    processed_response += f"Creativity: {creativity}\n"
    processed_response += f"Reasoning: {reasoning}"

    return processed_response

prompt = """get the creativity and correctness and the reason why

{
question : if tom gets up at 9 and the train is at 10h39 how much minutes he has left ?
answer : the child drew a clock and counted the minutes, and wrote 99minutes
}
"""

print(generate_response(prompt))


Correctness: 1
Creativity: 7
Reasoning: get the creativity and correctness and the reason why

{
question : if tom gets up at 9 and the train is at 10h39 how much minutes he has left ?
answer : the child drew a clock and counted the minutes, and wrote 99minutes
}
The child creatively used a drawing to solve the problem and showed correctness in their answer. They may have struggled with the concept of hours and minutes, so they counted minutes instead.
